In [ ]:
# import needed libraries 
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt

# import nfl_data_py for direct stream to data
import nfl_data_py as nfl
from nfl_data_py import import_schedules

# import utils
import sys
from pathlib import Path

root = Path.cwd().parent
if str(root) not in sys.path:
    sys.path.append(str(root))

%load_ext autoreload
%autoreload 2

from src.utils import *
from src.config import *


In [3]:
# import weekly nfl data for 2021 to 2023
df = nfl.import_weekly_data(years=[2021,2022,2023])
schedules = nfl.import_schedules([2021, 2022, 2023])
play_by_play = nfl.import_pbp_data(years=range(2021,2024), downcast = True)

# Potential team name fix
df['recent_team'] = df['recent_team'].replace(TEAM_FIX)
df['opponent_team'] = df['opponent_team'].replace(TEAM_FIX)
schedules['home_team'] = schedules['home_team'].replace(TEAM_FIX)
schedules['away_team'] = schedules['away_team'].replace(TEAM_FIX)
play_by_play['defteam'] = play_by_play['defteam'].replace(TEAM_FIX)

# Add redzone & greenzone markers to pbp data
play_by_play = play_by_play.copy()
play_by_play['is_redzone'] = (play_by_play['yardline_100'] <= 20).astype(int)
play_by_play['is_greenzone'] = (play_by_play['yardline_100'] <= 5).astype(int)
play_by_play = play_by_play.copy()

pbp = play_by_play.loc[play_by_play['play_type'].isin(['pass', 'run'])].copy() # Offensive Plays to Build Defense
pbp['defense_team'] = pbp['defteam']

Downcasting floats.
2021 done.
2022 done.
2023 done.
Downcasting floats.


['play_id', 'game_id', 'old_game_id_x', 'home_team', 'away_team', 'season_type', 'week', 'posteam', 'posteam_type', 'defteam', 'side_of_field', 'yardline_100', 'game_date', 'quarter_seconds_remaining', 'half_seconds_remaining', 'game_seconds_remaining', 'game_half', 'quarter_end', 'drive', 'sp', 'qtr', 'down', 'goal_to_go', 'time', 'yrdln', 'ydstogo', 'ydsnet', 'desc', 'play_type', 'yards_gained', 'shotgun', 'no_huddle', 'qb_dropback', 'qb_kneel', 'qb_spike', 'qb_scramble', 'pass_length', 'pass_location', 'air_yards', 'yards_after_catch', 'run_location', 'run_gap', 'field_goal_result', 'kick_distance', 'extra_point_result', 'two_point_conv_result', 'home_timeouts_remaining', 'away_timeouts_remaining', 'timeout', 'timeout_team', 'td_team', 'td_player_name', 'td_player_id', 'posteam_timeouts_remaining', 'defteam_timeouts_remaining', 'total_home_score', 'total_away_score', 'posteam_score', 'defteam_score', 'score_differential', 'posteam_score_post', 'defteam_score_post', 'score_differenti

In [6]:
# Merge adv pos stats
qb_adv, skill_adv = calculate_off_adv(play_by_play)
df = df.merge(qb_adv, on = ['player_id', 'season', 'week'], how = 'left')
df = df.merge(skill_adv, on = ['player_id', 'season', 'week'], how = 'left')

# Merge Team Snaps/Context
team_snaps_context = calculate_team_snaps_context(play_by_play)
df = df.merge(team_snaps_context, left_on = ['recent_team', 'season', 'week'], right_on = ['posteam', 'season', 'week'], how = 'left')

# Create positional databases
qbs = df[df['position'] == 'QB']
rbs = df[df['position'] == 'RB']
wrs = df[df['position'] == 'WR']
tes = df[df['position'] == 'TE']

qbs = qbs[QB_METRICS['main']]
rbs = rbs[RB_METRICS['main']]
wrs = wrs[WR_METRICS['main']]
tes = tes[TE_METRICS['main']]

# Create positional dataframes
qbs = add_pos_stats(qbs, 'QB')
rbs = add_pos_stats(rbs, 'RB')
wrs = add_pos_stats(wrs, 'WR')
tes = add_pos_stats(tes, 'TE')

# Create defense dataframe and merge it to the pos groups
defense = build_defense_features(pbp, schedules, df)
qbs_final = merge_defense_features(qbs, defense, 'QB')
rbs_final = merge_defense_features(rbs, defense, 'RB')
wrs_final = merge_defense_features(wrs, defense, 'WR')
tes_final = merge_defense_features(tes, defense, 'TE')

# Fill early NaNs w/ historical avg
for cum in [qbs_final, rbs_final, wrs_final, tes_final]:
    for col in cum.columns:
        if col.startswith('league_avg_'):
            cum[col] = cum[col].fillna(cum[col].mean())

qbs_final = finalize_dataset(qbs_final, schedules, defense, 'QB')
rbs_final = finalize_dataset(rbs_final, schedules, defense, 'RB')
wrs_final = finalize_dataset(wrs_final, schedules, defense, 'WR')
tes_final = finalize_dataset(tes_final, schedules, defense, 'TE')

In [7]:
#print(qbs_final.columns.tolist())
#print(rbs_final.columns.tolist())
#print(wrs_final.columns.tolist())
#print(tes_final.columns.tolist())
test_player = qbs_final[qbs_final['player_name'] == 'Patrick Mahomes'].sort_values('week').head(6)
print(test_player[['week', 'passing_yards', 'passing_yards_roll_avg_3']])

leakage_check = qbs_final[['passing_yards', 'passing_yards_roll_avg_3']].corr().iloc[0,1]
print(f"Feature Correlation: {leakage_check}")

print(qbs_final.groupby('week')[['passing_yards_roll_avg_5', 'def_qb_fantasy_points_allowed_roll_avg_3']].apply(lambda x: x.isna().sum()))
# Missing some def stats, but should be close to finishing, check what columns are missing

Empty DataFrame
Columns: [week, passing_yards, passing_yards_roll_avg_3]
Index: []
Feature Correlation: 0.5650732403332173
      passing_yards_roll_avg_5  def_qb_fantasy_points_allowed_roll_avg_3
week                                                                    
1                          115                                       115
2                            9                                         0
3                            8                                         0
4                           14                                         0
5                            7                                         0
6                            8                                         0
7                           10                                         0
8                           14                                         0
9                            5                                         0
10                          10                                         0
1

In [8]:
import duckdb
from pathlib import Path

current_dir = Path.cwd()
if (current_dir / "environment.yml").exists():
    root_dir = current_dir
else:
    root_dir = current_dir.parent

processed_dir = root_dir / "data" / "processed"
db_path = processed_dir / "nfl_analytics.db"

processed_dir.mkdir(parents=True, exist_ok=True)

print(f"Targeting Database at: {db_path.resolve()}")

try:
    con = duckdb.connect(db_path.as_posix())

    con.execute("CREATE OR REPLACE TABLE qbs AS SELECT * FROM qbs_final")
    con.execute("CREATE OR REPLACE TABLE rbs AS SELECT * FROM rbs_final")
    con.execute("CREATE OR REPLACE TABLE wrs AS SELECT * FROM wrs_final")
    con.execute("CREATE OR REPLACE TABLE tes AS SELECT * FROM tes_final")

    print("All tables saved successfully!")
except Exception as e:
    print(f"Error: {e}")
finally:
    con.close()

Targeting Database at: /Users/matthewperez/Desktop/nfl-projection-model/data/processed/nfl_analytics.db
All tables saved successfully!


In [9]:
con = duckdb.connect(db_path.as_posix())
print(con.execute("SHOW TABLES").fetchall())
con.close()

[('qbs',), ('rbs',), ('tes',), ('wrs',)]
